In [2]:
import os 
import glob
import sys
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


In [2]:
BASE_DIR = os.path.dirname(os.path.abspath('.'))+"\\data\\PostCorona\\"
card = pd.read_csv(BASE_DIR + "card.csv")

df = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202001.csv' , encoding = 'CP949')
fe = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202002.csv' , encoding = 'CP949')
mar = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202003.csv' , encoding = 'CP949')
ap = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202004.csv' , encoding = 'CP949')
may = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202005.csv' , encoding = 'CP949')
ju = pd.read_csv('C:/Users/park/Desktop/post_COVID_project/도로공사수도권교통량/202006.csv' , encoding = 'CP949')

In [3]:
traffic = [df,fe,mar,ap,may,ju]
traffic = pd.concat(traffic)
traffic.head()

,집계일자,집계시,영업소코드,입출구구분코드,TCS하이패스구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량,Unnamed: 12
0,20200101,0,101,0,1,316,6,3,9,1,5,340,NaN
1,20200101,0,101,0,2,1290,59,185,33,15,59,1641,NaN
2,20200101,0,101,1,1,263,13,3,13,4,25,321,NaN
3,20200101,0,101,1,2,1180,51,75,37,14,44,1401,NaN
4,20200101,0,190,0,1,189,3,0,5,1,2,200,NaN


In [4]:
# 데이터 컬럼 여러개 삭제하기
# axis = 0 (행기준)
# axis = 1(열기준)
traffic.drop(['집계시','1종교통량', '2종교통량', '3종교통량', '4종교통량',
       '5종교통량', '6종교통량','Unnamed: 12'], axis=1, inplace=True)

In [5]:
traffic.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량
0,20200101,101,0,1,340
1,20200101,101,0,2,1641
2,20200101,101,1,1,321
3,20200101,101,1,2,1401
4,20200101,190,0,1,200


In [6]:
# 집계일자를 인덱스로 바꾸자 datetime으로 변경하고
#df['receipt_dttm'] = pd.to_datetime(df['receipt_dttm'], format='%Y%m%d')
traffic['집계일자'] = pd.to_datetime(traffic['집계일자'], format='%Y%m%d')
traffic.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량
0,2020-01-01,101,0,1,340
1,2020-01-01,101,0,2,1641
2,2020-01-01,101,1,1,321
3,2020-01-01,101,1,2,1401
4,2020-01-01,190,0,1,200


In [7]:
traffic.set_index('집계일자',inplace=True)
traffic.head()

,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량
집계일자,,,,
2020-01-01,101,0,1,340
2020-01-01,101,0,2,1641
2020-01-01,101,1,1,321
2020-01-01,101,1,2,1401
2020-01-01,190,0,1,200


# 서울 톨게이트 = 101

In [8]:
# 서울톨게이트
# 컬럼과 컬럼 값 입력
# 월별로 구분해서 해야함
영업소코드 = 101
입출구구분코드 = 1
tmp = traffic[traffic['입출구구분코드']==입출구구분코드]
tmp = tmp[tmp["영업소코드"]==영업소코드]
tmp = tmp.reset_index()
tmp["영업소코드"] = 영업소코드
tmp["입출구구분코드"] = 입출구구분코드
tmp = tmp[["집계일자","영업소코드","입출구구분코드","총교통량"]]
tmp.head()

,집계일자,영업소코드,입출구구분코드,총교통량
0,2020-01-01,101,1,321
1,2020-01-01,101,1,1401
2,2020-01-01,101,1,384
3,2020-01-01,101,1,1460
4,2020-01-01,101,1,301


In [9]:
tmp['집계일자'] = pd.to_datetime(tmp['집계일자'], format='%Y%m%d')

In [10]:
tmp.set_index('집계일자', inplace=True)

In [11]:
# 주말빼고 주중만
tmp = tmp['총교통량'].resample('B').sum()
tmp.head()

집계일자
2020-01-01     80131
2020-01-02     99561
2020-01-03    293465
2020-01-06    100290
2020-01-07     93680
Freq: B, Name: 총교통량, dtype: int64

In [13]:
tmp = Series.to_frame(tmp)
tmp.head()

,총교통량
집계일자,
2020-01-01,80131
2020-01-02,99561
2020-01-03,293465
2020-01-06,100290
2020-01-07,93680


In [14]:
tmp = tmp.reset_index()

In [15]:
tmp.head()

,집계일자,총교통량
0,2020-01-01,80131
1,2020-01-02,99561
2,2020-01-03,293465
3,2020-01-06,100290
4,2020-01-07,93680


# 동서울 톨게이트 = 190

In [16]:
# 동서울톨게이트
# 컬럼과 컬럼 값 입력
# 월별로 구분해서 해야함
영업소코드 = 190
입출구구분코드 = 1
tmp1 = traffic[traffic['입출구구분코드']==입출구구분코드]
tmp1 = tmp1[tmp1["영업소코드"]==영업소코드]
tmp1 = tmp1.reset_index()
tmp1["영업소코드"] = 영업소코드
tmp1["입출구구분코드"] = 입출구구분코드
tmp1 = tmp1[["집계일자","영업소코드","입출구구분코드","총교통량"]]
tmp1.head()

,집계일자,영업소코드,입출구구분코드,총교통량
0,2020-01-01,190,1,132
1,2020-01-01,190,1,640
2,2020-01-01,190,1,128
3,2020-01-01,190,1,653
4,2020-01-01,190,1,104


In [18]:
tmp1['집계일자'] = pd.to_datetime(tmp1['집계일자'], format='%Y%m%d')

In [19]:
tmp1.set_index('집계일자', inplace=True)

In [20]:
# 주말빼고 주중만
tmp1 = tmp1['총교통량'].resample('B').sum()
tmp1.head()

집계일자
2020-01-01     55139
2020-01-02     61760
2020-01-03    191857
2020-01-06     61517
2020-01-07     56060
Freq: B, Name: 총교통량, dtype: int64

In [21]:
tmp1 = Series.to_frame(tmp1)
tmp1.head()

,총교통량
집계일자,
2020-01-01,55139
2020-01-02,61760
2020-01-03,191857
2020-01-06,61517
2020-01-07,56060


In [22]:
tmp1 = tmp1.reset_index()

In [23]:
tmp1.head()

,집계일자,총교통량
0,2020-01-01,55139
1,2020-01-02,61760
2,2020-01-03,191857
3,2020-01-06,61517
4,2020-01-07,56060


# 서서울 톨게이트 = 253

In [24]:
# 동서울톨게이트
# 컬럼과 컬럼 값 입력
# 월별로 구분해서 해야함
영업소코드 = 253
입출구구분코드 = 1
tmp2 = traffic[traffic['입출구구분코드']==입출구구분코드]
tmp2 = tmp2[tmp2["영업소코드"]==영업소코드]
tmp2 = tmp2.reset_index()
tmp2["영업소코드"] = 영업소코드
tmp2["입출구구분코드"] = 입출구구분코드
tmp2 = tmp2[["집계일자","영업소코드","입출구구분코드","총교통량"]]
tmp2.head()

,집계일자,영업소코드,입출구구분코드,총교통량
0,2020-01-01,253,1,214
1,2020-01-01,253,1,902
2,2020-01-01,253,1,238
3,2020-01-01,253,1,1009
4,2020-01-01,253,1,173


In [25]:
tmp2['집계일자'] = pd.to_datetime(tmp2['집계일자'], format='%Y%m%d')

In [26]:
tmp2.set_index('집계일자', inplace=True)

In [27]:
# 주말빼고 주중만
tmp2 = tmp2['총교통량'].resample('B').sum()
tmp2.head()

집계일자
2020-01-01     70103
2020-01-02     85842
2020-01-03    252206
2020-01-06     86042
2020-01-07     81426
Freq: B, Name: 총교통량, dtype: int64

In [28]:
tmp2 = Series.to_frame(tmp2)
tmp2.head()

,총교통량
집계일자,
2020-01-01,70103
2020-01-02,85842
2020-01-03,252206
2020-01-06,86042
2020-01-07,81426


In [29]:
tmp2 = tmp2.reset_index()

In [30]:
tmp2.head()

,집계일자,총교통량
0,2020-01-01,70103
1,2020-01-02,85842
2,2020-01-03,252206
3,2020-01-06,86042
4,2020-01-07,81426
